In [1]:
# Loading required packages and libraries
import pandas as pd
import matplotlib.pyplot as plt
import warnings

In [2]:
# Setting default options
pd.set_option("mode.copy_on_write", True)
warnings.simplefilter(action="ignore", category=FutureWarning)

In [3]:
# Loading required  data
df = pd.read_csv("../01_Data/01_Raw/nhis.csv.zip", compression="zip")
df.sample(5)

/var/folders/cj/jkp7xyld4_q3x3rk7xdx7jwh0000gn/T/ipykernel_84273/493222604.py:2: DtypeWarning: Columns (4,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../01_Data/01_Raw/nhis.csv.zip", compression="zip")


,YEAR,SERIAL,STRATA,PSU,NHISHID,HHWEIGHT,REGION,PERNUM,NHISPID,HHX,...,SMKECIGRND,CIGAREV,CIGARDAYSMO,CIGAR50LIFE,CIGARFREQNOW,PIPEV,PIPEFREQNOW,NONCIGEV,NONCIGNOW,TOBLSYR
436258,2004,21857,5221,1,2004036878,3315.0,2,4,20040368780104,36878,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1197700,2013,1773,6266,2,2013002634,562.0,4,2,20130026340102,2634,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
293111,2002,39630,5093,1,2002065998,1023.0,2,2,20020659980102,65998,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1255756,2013,31730,6279,2,2013047119,679.0,4,1,20130471190101,47119,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,2.0
1494324,2015,47097,6070,1,2015068259,2461.0,2,7,20150682590107,68259,...,0.0,0.0,96.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN


In [4]:
# check values in CNLUNG
df["CNLUNG"].value_counts()

CNLUNG
0    1849834
1      61839
2       2049
9        238
7        105
Name: count, dtype: int64

In [5]:
# Subsetting the required columns, filternring for CNLUNG as 1 or 2
df_2 = df.loc[
    df["CNLUNG"].isin([1, 2]), ["YEAR", "CNLUNG", "ALCSTAT1", "SMOKESTATUS2", "SEX"]
]
df_2.sample(5)

,YEAR,CNLUNG,ALCSTAT1,SMOKESTATUS2,SEX
1614961,2017,1,3.0,20,2
1710300,2018,1,2.0,30,1
1648196,2017,1,1.0,30,2
1576579,2016,1,3.0,30,2
272829,2002,1,9.0,30,2


In [15]:
# New Column to encode alcohol status
def map_alc(x):
    if x == 1:
        return "No"
    elif x == 2 or x == 3:
        return "Yes"
    else:
        return "Other"


df_2["ALC"] = df_2["ALCSTAT1"].apply(map_alc)

In [16]:
df_2["ALC"].value_counts()

ALC
Yes      45490
No        9978
Other     8420
Name: count, dtype: int64

In [17]:
# Proportion of CNLUNG=1 by ALCSTAT1 year wise, line plot
proportion_by_alc_year = (
    df_2.loc[df_2["ALC"].isin(["Yes", "No"]), :]
    .groupby(["YEAR", "ALC"])["CNLUNG"]
    .apply(lambda x: (x == 2).mean())
    .unstack()
)
proportion_by_alc_year

ALC,No,Yes
YEAR,,
2000,0.023529,0.023632
2001,0.017204,0.036364
2002,0.040169,0.035138
2003,0.050526,0.038913
2004,0.028736,0.033391
2005,0.028143,0.028007
2006,0.032520,0.035633
2007,0.033058,0.040411
2008,0.029268,0.026580
